In [35]:
#import dependencies
import os
import pandas as pd
import glob

In [40]:
#I want to display the data to better understand what I am working with
#path = 'low_freq/house_1/'
#forward slash to access path and a one more slash to access the folder of the file
path = 'E:/Document/Master/Dataset/Energy_disaggregation/REDD_dataset/low_freq/house_1/' 
file = path + 'channel_1.dat'
df = pd.read_table(file, sep= ' ')
print(df)

1303132929  224.19
0        1303132930  225.57
1        1303132931  226.09
2        1303132932  222.74
3        1303132933  222.20
4        1303132934  222.11
...             ...     ...
1561654  1306267018  234.44
1561655  1306267019  235.57
1561656  1306267020  234.77
1561657  1306267021  235.55
1561658  1306267022  235.43

[1561659 rows x 2 columns]
